In [360]:
import numpy as np
import pandas as pd 
from pylab import mpl, plt
import random
import optuna
import os
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
torch.set_num_threads(1)


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

import warnings
warnings.simplefilter("ignore", UserWarning)

import vectorbtpro as vbt


from collections import deque

vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 800
vbt.settings['plotting']['layout']['height'] = 400


In [361]:
df_sol = pd.read_csv('data/1ySOLdata1hAllHassInd.csv')
df_sol['timestamp'] = pd.to_datetime(df_sol['timestamp'], unit='s')
df_sol.set_index('timestamp', inplace=True)
# pd.set_option('future.no_silent_downcasting', True)

In [362]:
# data = df_sol.iloc[:, 3:4].copy()
# data_trimmed = df_sol[-5800:-4900].copy()
data_trimmed = df_sol.copy()

In [363]:
pd.set_option('future.no_silent_downcasting', True)
data_trimmed.loc[:, 'signal'] = 'SignalNone'
# Define window size
window_size = 30

rolling_max = data_trimmed.loc[:,'Close'].rolling(window=2*window_size+1, center=True, min_periods=1).max()
rolling_min = data_trimmed.loc[:,'Close'].rolling(window=2*window_size+1, center=True, min_periods=1).min()

is_peak = (data_trimmed.loc[:, 'Close'] == rolling_max)

is_low = (data_trimmed.loc[:, 'Close'] == rolling_min) 

# Update signal columns where conditions are met
data_trimmed.loc[is_peak, 'signal'] = 'SignalShort'  # Mark peaks with SignalShort
data_trimmed.loc[is_low, 'signal'] = 'SignalLong'   # Mark lows with SignalLong
df = data_trimmed.copy()

In [364]:
def filter_pivots(data):
    df_filtered = data[data['signal'] != 'SignalNone'].copy()

    for i in range(1, len(df_filtered)):
        current_signal = df_filtered.iloc[i]['signal']
        previous_signal = df_filtered.iloc[i - 1]['signal']
        current_close = df_filtered.iloc[i]['Close']
        previous_close = df_filtered.iloc[i - 1]['Close']
        
        if current_signal == previous_signal:
            if current_signal == 'SignalLong':
                if previous_close > current_close:
                    df_filtered.iloc[i - 1, df_filtered.columns.get_loc('signal')] = 'SignalNone'
                else:
                    df_filtered.iloc[i, df_filtered.columns.get_loc('signal')] = 'SignalNone'
            elif current_signal == 'SignalShort':
                if previous_close < current_close:
                    df_filtered.iloc[i - 1, df_filtered.columns.get_loc('signal')] = 'SignalNone'
                else:
                    df_filtered.iloc[i, df_filtered.columns.get_loc('signal')] = 'SignalNone'

    return df_filtered

filter_1 = filter_pivots(df)

df.update(filter_1)
next_filter = df[['Close', 'signal']].copy()

filter_2 = filter_pivots(next_filter)
df.update(filter_2)

In [365]:
df_fixed = df.copy()
df_fixed.loc[:,'signal'] = df_fixed.loc[:,'signal'].replace({'SignalLong': 2, 'SignalShort': 0, 'SignalNone': 1})
df_fixed = df_fixed.ffill()

In [366]:
# df['signal'] = df['signal'].astype(float)
# long_signals = df['signal'] == 2
# short_signals = df['signal'] == 0

In [367]:
# scaler = MinMaxScaler(feature_range=(0, 2))
# for idx in df.index[:-1]:
#     if short_signals.loc[idx]:
#         short_index = idx
#         next_long_idx = df.loc[idx:].index[long_signals[idx:]].min()
#         bear_slice = df.loc[short_index : next_long_idx].copy()
#         bear_slice['signal'] = bear_slice['Close']
#         signal_values = bear_slice['signal'].values.reshape(-1, 1)
#         scaled_signal_values = scaler.fit_transform(signal_values)
#         scaled_signal_values_transformed = 2 - (scaled_signal_values)
#         bear_slice['signal'] = scaled_signal_values_transformed.flatten()
#         df.update(bear_slice)
#     elif long_signals.loc[idx]:
#         long_index = idx
#         next_short_idx = df.loc[idx:].index[short_signals[idx:]].min()
#         bull_slice = df.loc[long_index : next_short_idx].copy()
#         bull_slice['signal'] = bull_slice['Close']
#         signal_values = bull_slice['signal'].values.reshape(-1, 1)
#         scaled_signal_values = scaler.fit_transform(signal_values)
#         scaled_signal_values_transformed = 2 - (scaled_signal_values)
#         bull_slice['signal'] = scaled_signal_values_transformed.flatten()
#         df.update(bull_slice)


In [368]:
signal = df_fixed['signal']
entries = signal == 2
exits = signal == 0
pf = vbt.Portfolio.from_signals(
    close=df_fixed.Close, 
    long_entries=entries, 
    short_entries=exits,
    size=100,
    size_type='value',
    accumulate=True,
    init_cash='auto'
)
pf.plot({"orders"}).show()




In [369]:
# fig = signal.vbt.plot()
# fig.update_layout(yaxis_title='signal')
# fig.show()